# Projet Recherche Opérationelle - Branch And Bound

## Introduction



Lorem ipsum dolor sit amet, consectetur adipiscing elit. Vivamus consectetur congue nisl, ac laoreet erat mattis vitae. Donec vestibulum vehicula commodo. Class aptent taciti sociosqu ad litora torquent per conubia nostra, per inceptos himenaeos. Nam nec leo sit amet augue maximus aliquam a id urna. Nam rutrum dictum nulla, in cursus sapien pretium eu. Sed mollis tincidunt nisi at rutrum. Maecenas mauris turpis, gravida tristique pretium a, porta sit amet dolor. Vestibulum vehicula vestibulum tortor ultrices interdum. Praesent vel placerat ipsum, a maximus erat. Nam ac cursus sem. Quisque faucibus tortor venenatis aliquam congue. Vivamus faucibus lacus sed dolor mollis, ac posuere tortor placerat. Donec pellentesque et ligula gravida semper. Quisque cursus ante arcu, vehicula sodales urna imperdiet et. Nunc commodo porttitor interdum. Curabitur ultrices magna ac urna vulputate, vitae pellentesque libero viverra.

Sed eget orci a erat interdum ultrices. Pellentesque est nunc, cursus a rhoncus at, molestie ut justo. Integer ut placerat sapien, sit amet convallis turpis. Curabitur tincidunt tempor iaculis. Cras viverra dignissim purus quis rutrum. Quisque sagittis commodo est id laoreet. Nam varius blandit cursus. Praesent venenatis ultrices nisi ut maximus. Vivamus sed facilisis turpis.

Mauris fermentum lorem vel tincidunt ultrices. Morbi nibh nunc, iaculis ac scelerisque dapibus, malesuada ac ex. Curabitur fermentum interdum felis id aliquet. Cras elementum leo eu accumsan accumsan. Integer vitae aliquet nisl. Pellentesque habitant morbi tristique senectus et netus et malesuada fames ac turpis egestas. Morbi porta libero vitae fermentum convallis. Donec sagittis id sapien et vestibulum. Suspendisse mattis orci a risus egestas, accumsan hendrerit lacus egestas. Proin non purus at lorem tempus fermentum eget ac ipsum. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus. In et urna tincidunt eros feugiat rhoncus id sit amet lectus.

Sed sit amet sem tristique, mollis sapien quis, lacinia augue. Sed lorem neque, molestie iaculis imperdiet eu, luctus ac massa. Fusce at augue ut lectus pretium dictum ut ut lacus. Praesent egestas nulla sodales leo porta, eu dapibus dolor pellentesque. Vivamus venenatis accumsan tristique. Vestibulum pellentesque nisi quis massa tincidunt molestie. Sed gravida augue vel lorem blandit, tristique sollicitudin quam aliquam. Ut fermentum rhoncus justo non porttitor. Praesent condimentum lorem in risus porta, eget convallis est dictum. Etiam ac nulla nec ex euismod porttitor. Fusce sed dui porta, faucibus augue at, consectetur eros. Proin suscipit rutrum mi vel ullamcorper. Pellentesque placerat nisi est, eget blandit eros maximus a. Maecenas nulla lacus, pharetra interdum molestie quis, tincidunt ut justo. Cras ultricies mauris eget vehicula accumsan. Orci varius natoque penatibus et magnis dis parturient montes, nascetur ridiculus mus.

Phasellus mattis tellus ante, et mattis turpis pharetra et. Quisque tristique ex et dolor varius ullamcorper. Etiam neque quam, elementum eget neque ac, congue bibendum lorem. Nunc dictum consequat turpis. Pellentesque dolor sapien, gravida quis condimentum egestas, ultricies id augue. Nunc velit lectus, tincidunt eget lacinia sed, finibus quis odio. Interdum et malesuada fames ac ante ipsum primis in faucibus. Cras mattis et massa id aliquam. Suspendisse ornare condimentum varius. 

## Importation des modules

Nous importons tout d'abord tous les modules nécessaires dans ce projet. Nous utilisons les modules classiques __numpy__ et __matplotlib (pyplot)__. Nous utilisons égalements les modules __networkx__ et un module particulier de pydot (cellule pour installation ci dessous) lors de la visualisation de parcours de l'arbre de branchement.

In [4]:
conda install pydot

Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [8]:
import numpy as np
import math
import networkx as nx
from networkx.drawing.nx_pydot import graphviz_layout
from matplotlib import pyplot as plt
import copy

AttributeError: module 'networkx' has no attribute 'utils'

## Structure de données

*Nous définissons un certain nombre de structures de données, primordiales pour l'implémentation de l'algorithme de Branch and Bound.*

### Les noeuds 

Chaque noeud de l'arbre de branchement sera un objet. Chaque objet Noeud créé possède :
* Un indexe : __idx__
* Un noeud parent : __parent__
* Une liste de noeuds enfants : __children__
* Une borne duale associée (lower bound ici, puisqu'il s'agit d'un problème de minimisation) : __lb__
* Une liste de noeuds visités : __visited__.
* Une méthode d'affichage.

Dans cette première partie, les index de noeuds seront des entiers, leur stockage sera rapide. Dans la partie __affichage__ les index seront des chaines de caractères plus ou moins longues, leur stockage sera légèrement plus long.

__Exemple__

![Arbre de branchement](EX2.png)

En considérant que les noeud en vert on déjà été visités et que les noeuds rouges ne l'ont pas été, le noeud 1 représenté ci dessus possèderait (selon la structure choisie) les éléments suivants : 
* __parent__=0
* __children__=[ 4 , 5 ]
* __visited__=[ 0 ]

In [6]:
class Node:
    
    def __init__(self,idx, parent, children, lb, visited):
        self.idx = idx
        self.parent = parent
        self.children = children
        self.lb = lb
        self.visited = visited
        
    def __str__(self):
        return ("Parent : " +str(self.parent)+
                "\n Children : "+str(self.children) + 
                "\n lb : "+str(self.lb)+ 
                "\n visited : "+str(self.visited))

### Fonctions abroresence et computeCost

*Nous implémentons à présent un certain nombre de fonctions que nous utiliserons lors de l'implémentation de l'algorithme de branch and bound.*

La première fonction est une fonction arborescence. Elle nous permet de donner une branche complète d'arbre de branchement en partant d'une feuille. Dans l'__exemple__ ci-dessus, la fonction : __arborescence(4,Tree)__ renverrait : [4,1,0]. 

In [9]:
def arborescence(node,Tree):
    branch = []
    while node.parent != None:
        branch.append(node.idx)
        node = node.parent
    branch.append(node.idx)
    
    return branch

Afin de déterminer facilement le cout d'une solution donnée, nous implémentons une méthode générale computeCost. A partir des données du problème (p,d,w) et d'un ordre de taches à effectuer, la fonction renvoie le cout de la solution.

In [10]:
def computeCost(p,d,w,order):
    t=0
    cost=0
    for idx in order:
        t += p[idx]
        cost += max(0,(t-d[idx])*w[idx])
    return cost

Afin de tester la dernière méthode nous utilisons le problème vu en TD (*exercice 7, TD branch and bound*) dont les instances sont rappelées ci-dessous : 

* unités de temps pour que les pièces soient usinées : p=[12,8,15,9]
* date auxquelles les pièces doivent être prêtes : d=[16,26,25,27]
* pénalités de retard : w=[4,5,3,5]

In [12]:
p = [12,8,15,9]
d = [16,26,25,27]
w = [4,5,3,5]

La solution que nous testons (ordre d'usinage : [0,1,3,2]) est la solution optimale, obtenue lors du TD. Son coût vaut 67. On vérifie donc que la fonction computeCost nous donne bien 67 avec cet ordre d'usinage.

In [13]:
computeCost(p,d,w,[0,1,3,2])

67

### Calculs de bornes primales

L'idée de ce projet est de tester différentes composantes de l'algorithme de Branch and Bound, afin de voir leur impact sur les performances de l'algorithme. 

Nous nous concentrons tout d'abord sur le calcul des bornes primales. Le calcul de borne primale s'effectue au début de l'algorithme. Plus le calcul est fin, i.e plus la borne primale obtenue possède une valeur faible, plus l'algorithme ira vite. L'idée est donc d'essayer de trouver la méthode la plus interessante.


Nous avons fait le choix de coder différentes fonctions :
* __getPrimal_0__ : nous calculons une solution en usinant les pièces selon l'ordre croissant des dates d'usinage au plus tard.
* __getPrimal_1__ : nous calculons une solution en usinant les pièces selon l'ordre croissant des (dates d'usinage au plus tard-le temps d'usinage)$*$pénalité

Comme les premières pièces engendrent le moins de retard, nous nous sommes dit qu'il est possible d'obtenir une meilleure borne primale en réflechissant à la position des derniers éléments. Cela nous a amené à la fonction : 
* __getPrimal_2__ : nous calculons une solution en mettant à la dernière position disponible l'élément non traité donnant lieu à la plus petite pénalité de retard.

In [14]:
def getPrimal_0(p,d,w):
    sortedTasks=np.argsort(d)
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,sortedTasks)


def getPrimal_1(p,d,w):
    sortedTasks=(np.argsort((np.array(d)-np.array(p))*np.array(w)))[::-1]
    cost=computeCost(p,d,w,sortedTasks)
    return(cost,sortedTasks)


def getPrimal_2(p,d,w):
    tasksLeft=[int(i) for i in range(len(p))]
    order=[]
    while len(tasksLeft)!= 0:
        time=np.sum(np.array(p)[tasksLeft])
        penalities=(time-np.array(d)[tasksLeft])*np.array(w)[tasksLeft]
        sortedTasks=np.argsort(penalities)
        order.append(tasksLeft[sortedTasks[0]])
        tasksLeft.pop(sortedTasks[0])
    order.reverse()
    cost = computeCost(p,d,w,order)
    return(cost,order)

Nous comparons à présent les résultats de ces méthodes en utilisant les instances p,d,w initialisée ci dessus (*EX7 TD branch and bound*). __Rappel__: plus une fonction de calcul de borne duale est performante, plus la valeur de la borne duale trouvée est petite.

In [15]:
print("Temps production : ", p)
print("Date limite :      ", d)
print("Pénalités :        ", w)
print("0 : ",getPrimal_0(p,d,w))
print("1 : ",getPrimal_1(p,d,w))
print("2 : ",getPrimal_2(p,d,w))

Temps production :  [12, 8, 15, 9]
Date limite :       [16, 26, 25, 27]
Pénalités :         [4, 5, 3, 5]
0 :  (136, array([0, 2, 1, 3]))
1 :  (133, array([3, 1, 2, 0]))
2 :  (67, [0, 1, 3, 2])


__Conclusion :__ la méthode getPrimal_2 est bien plus performante que les méthodes 0 et 1.

### Calcul des bornes duales [A REDIGER]

In [16]:
def getDual_0(p,d,w,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    for j in notVisited:
        retard=(p[j]+time)-d[j]
        dual+=retard*w[j]*(retard>0)
    return dual

def getDual_1(p,d,w,visited):
    dual=0
    time=0
    notVisited=list(range(len(p)))
    for i in visited:
        retard=((p[i]+time)-d[i])
        dual+=retard*w[i]*(retard>0)
        time+=p[i]
        notVisited.remove(i)
        
    return dual

def getDual_2(p,d,w,visited):
    dual=0
    time=np.sum(p)

    for i in visited:
        retard=(time-d[i])
        dual+=retard*w[i]*(retard>0)
        time-=p[i]
        
    return dual

In [17]:
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : \n",
      getDual_0(p,d,w,[2,3]))
print("Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : \n",
      getDual_1(p,d,w,[2,3]))

Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces et on ajoute les pénalités des autres pièces en supposant qu'elles sont en (k+1)ème position : 
 110
Si on a fixé l'ordre de k pièces on calcule les pénalités de retard de ces k pièces : 
 0


### Méthodes d'exploration [A REDIGER]

In [18]:
def exploration_0(Queue,Tree,UB):
    return(Queue[0])

def exploration_1(Queue,Tree,UB):
    return(Queue[-1])

def exploration_2(Queue,Tree,UB):
    bestNode = None
    bestLB = -math.inf
    for i in range(len(Queue)):
        if Tree[Queue[i]].lb > bestLB and Tree[Queue[i]].lb < UB:
            bestLB = Tree[Queue[i]].lb
            bestNode = i
    return(Queue[i])

## Branch and Bound [A REDIGER + A COMPLETER]

In [19]:
###################
#  CODE GOES HERE #
##################

## Performances de l'algorithme de Branch and Bound [A REDIGER + A COMPLETER]

In [20]:
###################
#  CODE GOES HERE #
##################

## Visualisation de l'arbre de branchement.

L'idée est ici d'implémenter une fonction de visualisation de l'arbre de branchement. 

### Redéfinition de quelques éléments de structure

Comme évoqué aupravant, nous changeons les index des noeuds. Nous sommes conscients que stocker des entiers est moins couteux en temps de calcul que stocker des grandes chaines de caractères et c'est pour cette raison que nous avons fait une nouvelle version de la fonction branch and bound ci dessous. 

Afin de mieux comprendre notre facon d'indexer les noeuds, nous présentons un exemple succinct. 

![Arbre de branchement](EX1.png)

* Le noeud racine sera __P__
* Le noeud __P(1)__ signifie que la solution liée à ce noeud place l'usinage de la pièce 1 en dernier.
* Le noeud __P(2,1)__ signifie que la solution liée à ce noeud place l'usinage de la pièce 1 en dernier, puis celui de la pièce 2 en avant dernier.

Pour des exmples de taille plus conséquente (+ de taches), par exemple 5, certains des noeuds seront instanciés par __P(*4 chiffres séparés par des virgules*)__.

### Branch and bound redéfini

Nous utilisons ces nouvelles indexations pour mettre à jour la fonction branch and bound. 

Notons aussi que nous ajoutons quelques élements de code permettant de trouver la branche de l'arbre qui aura donné lieu à la meilleure solution. Pour cela, nous localisons le meilleur noeud, i.e le noeud qui :
* est tel que la borne duale du noeud est inférieure ou égale à la borne primale du problème                **(1)**
* est tel que la borne duale du noeud est plus grande que n'importe quel borne duale des noeuds vérifiant (1).

Nous remontons ensuite (en utilisant les parents) afin d'obtenir la branche de l'arbre donnant lieu à la meilleure solution. Cela nous sera utile lors de la visualisation de l'arbre de branchement.

Les éléments retournés par la fonction sont également ajustés. Nous ajoutons le retour de : 
* __branch__ : la branche donnant lieu à la solution optimale du problème
* __Tree__: l'arbre correspondant au problème

In [21]:
def branch_and_bound2(p,d,w,byLast,getPrimal,getDual,exploration):
    
    Tasks = list(range(len(p)))
    Tree = [] # list of all nodes created (a list of Node objects)
    Queue = [] # list of nodes to process (a list of integers with the index of nodes to process in the Tree)
    UB = 10000000 # set the upper bound to a sufficiently large number
    LB = 0 # set the lower bound to a sufficiently small number 
    
    UB,currentOrder = getPrimal(p,d,w)

    bestNode = Node("",-1, [], 0 ,[])
    
    root = Node("P",None, [], 0, []) # at the root node no variables are fixed 
    Tree.append(root) # start the tree with the root node
    Queue.append(0) # start the list of nodes to process with the root node
    
    #continue processing nodes until the queue is empty
    
    nbIterations = 0
    
    while Queue!=[]:
        
        # process the first node in the queue

        currentIndex = exploration(Queue,None,None)
        currentNode = Tree[currentIndex]
        nbIterations +=1
        
        if currentNode.lb < UB:
            
            if len(currentNode.visited) == len(p):
                order = currentNode.visited.copy()
                if byLast:
                    order.reverse()
                solutionCost = computeCost(p,d,w,order)
                if solutionCost <= UB:
                    UB = solutionCost
                    currentOrder=order
                    bestNode = currentNode
            else:
                for nextTask in (set(Tasks)-set(currentNode.visited)):
                    nextVisited = currentNode.visited.copy()
                    nextVisited.append(nextTask)
                    nextLb = getDual(p,d,w, nextVisited)

                    newNode = Node("",currentNode, [], nextLb,nextVisited)
                    
                    if currentNode.visited!=[]:
                        newNode.idx="P("
                        for i in range(len(nextVisited)-1):
                            newNode.idx+=str(nextVisited[i])+","
                        newNode.idx+=str(nextVisited[len(nextVisited)-1])+")"
                    else:
                        newNode.idx="P("+str(nextTask)+")"
                        
                    Tree.append(newNode)
                    Queue.append(len(Tree)-1)
                    currentNode.children.append(newNode)
            
        # When the processing of the node is completed remove the node from the queue     
        Queue.remove(currentIndex)

    # return the  solution
    
    if 2*(UB-bestNode.lb)<=0.0001:
        branch = arborescence(bestNode,Tree)
    else:
        bestLB = -math.inf
        for node in Tree:
            if node.lb > bestLB and node.lb <=UB:
                bestLB = node.lb
                bestNode=node
        branch = arborescence(bestNode,Tree)

    return UB, currentOrder, nbIterations, branch, Tree